In [2]:
 # Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

In [3]:
#Store Part I results into DataFrame
#Load the csv exported in Part I to a DataFrame

Weather_df = pd.read_csv("output_data_file.csv")
Weather_df.head()

,Unnamed: 0,City,City ID,Temperature (F),Humidity (%),Cloud Coverage (%),Wind Speed (mph),Latitude,Longitude
0,0,Avarua,4035715,78.80,100,65,2.24,-21.21,-159.78
1,1,College,5859699,24.42,85,75,4.70,64.86,-147.80
2,2,Hermanus,3366880,67.75,76,100,5.01,-34.42,19.23
3,3,Ushuaia,3833367,47.52,49,40,18.34,-54.80,-68.30
4,4,Praia da Vitória,3372760,64.40,72,20,11.77,38.73,-27.07


In [4]:
#Humidity Heatmap

#Configure gmaps.
#Use the Lat and Lng as locations and Humidity as the weight.
#Add Heatmap layer to map.

locations = Weather_df[["Latitude", "Longitude"]]
humidity = Weather_df["Humidity (%)"].astype(float)


fig = gmaps.figure()
heatmap_layer = gmaps.heatmap_layer(locations, weights=humidity,dissipating=False, max_intensity=5,point_radius=3)
fig.add_layer(heatmap_layer)
fig


Figure(layout=FigureLayout(height='420px'))

In [5]:
#Create new DataFrame fitting weather criteria
#Narrow down the cities to fit weather conditions. (A max temperature lower than 80 degrees but higher than 70,
#Wind speed less than 10 mph, Wind speed less than 10 mph.)

New_Cities = Weather_df.loc[(Weather_df["Temperature (F)"] > 70) 
                            & (Weather_df["Temperature (F)"] < 80) 
                            & (Weather_df["Cloud Coverage (%)"] == 0), :]

New_Cities = New_Cities.dropna(how='any')
New_Cities.reset_index(inplace=True)
del New_Cities['index']
New_Cities.head(12)

,Unnamed: 0,City,City ID,Temperature (F),Humidity (%),Cloud Coverage (%),Wind Speed (mph),Latitude,Longitude
0,95,Parlākimidi,1260296,79.47,75,0,3.65,18.77,84.08
1,230,Henties Bay,3356832,72.64,58,0,10.33,-22.12,14.28
2,232,Lüderitz,3355672,73.40,49,0,9.17,-26.65,15.16
3,253,Cockburn Town,3576994,79.41,71,0,4.47,21.46,-71.14
4,324,Aswān,359792,75.20,19,0,16.11,24.09,32.91
5,346,Santiago del Estero,3835869,79.41,37,0,11.79,-27.80,-64.26
6,425,Ubari,2219235,79.25,15,0,11.18,26.59,12.78
7,436,Bubaque,2374583,79.16,75,0,13.35,11.28,-15.83
8,440,Port Hedland,2063042,75.20,78,0,5.82,-20.32,118.57
9,469,Chapecó,3466296,75.99,35,0,3.58,-27.10,-52.62


In [6]:
#Hotel Map

#Store into variable named hotel_df.
#Add a "Hotel Name" column to the DataFrame.
#Set parameters to search for hotels with 5000 meters.
#Hit the Google Places API for each city's coordinates.
#Store the first Hotel result into the DataFrame.
#Plot markers on top of the heatmap.

Hotel = []

for i in range(len(New_Cities)):
    lat = New_Cities.loc[i]['Latitude']
    lng = New_Cities.loc[i]['Longitude']

    params = {
        "location": f"{lat},{lng}",
        "radius": 5000,
        "types" : "hotel",
        "key": g_key
    }
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    requested = requests.get(base_url, params=params)
    jsn = requested.json()
    try:
        Hotel.append(jsn['results'][0]['name'])
    except:
        Hotel.append("")
New_Cities["Hotel Name"] = Hotel
New_Cities

,Unnamed: 0,City,City ID,Temperature (F),Humidity (%),Cloud Coverage (%),Wind Speed (mph),Latitude,Longitude,Hotel Name
0,95,Parlākimidi,1260296,79.47,75,0,3.65,18.77,84.08,Paralakhemundi
1,230,Henties Bay,3356832,72.64,58,0,10.33,-22.12,14.28,Hentiesbaai
2,232,Lüderitz,3355672,73.40,49,0,9.17,-26.65,15.16,Luderitz
3,253,Cockburn Town,3576994,79.41,71,0,4.47,21.46,-71.14,Cockburn Town
4,324,Aswān,359792,75.20,19,0,16.11,24.09,32.91,Aswan
5,346,Santiago del Estero,3835869,79.41,37,0,11.79,-27.80,-64.26,Santiago del Estero
6,425,Ubari,2219235,79.25,15,0,11.18,26.59,12.78,Awbari‎
7,436,Bubaque,2374583,79.16,75,0,13.35,11.28,-15.83,Bubaque
8,440,Port Hedland,2063042,75.20,78,0,5.82,-20.32,118.57,Port Hedland
9,469,Chapecó,3466296,75.99,35,0,3.58,-27.10,-52.62,Chapecó


In [9]:
#Display the map
locations = New_Cities[["Latitude", "Longitude"]]
#locations

markers = gmaps.marker_layer(locations)
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))